In [1]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import re

In [2]:
data1 = pd.read_excel("/Users/jongwoom1pro/Coding/text_magician/MJ_newsdata/통계청_감성지수라벨링자료2020년01월_20201229.xlsx")
data2 = pd.read_excel("/Users/jongwoom1pro/Coding/text_magician/MJ_newsdata/통계청_감성지수라벨링자료2020년02월_20201229.xlsx")
data3 = pd.read_excel("/Users/jongwoom1pro/Coding/text_magician/MJ_newsdata/통계청_감성지수라벨링자료2020년03월_20201229.xlsx")
data4 = pd.read_excel("/Users/jongwoom1pro/Coding/text_magician/MJ_newsdata/통계청_감성지수라벨링자료2020년04월_20201229.xlsx")
data5 = pd.read_excel("/Users/jongwoom1pro/Coding/text_magician/MJ_newsdata/통계청_감성지수라벨링자료2020년05월_20201229.xlsx")
data6 = pd.read_excel("/Users/jongwoom1pro/Coding/text_magician/MJ_newsdata/통계청_감성지수라벨링자료2020년06월_20201229.xlsx")
data7 = pd.read_excel("/Users/jongwoom1pro/Coding/text_magician/MJ_newsdata/통계청_감성지수라벨링자료2020년07월_20201229.xlsx")
data8 = pd.read_excel("/Users/jongwoom1pro/Coding/text_magician/MJ_newsdata/통계청_감성지수라벨링자료2020년08월_20201229.xlsx")

# data concat
# 데이터프레임을 연결(concatenate)
concatenated_data = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8], ignore_index=True)

# 결과 확인
concatenated_data

,날짜,제목,URL,라벨러1,라벨러2,라벨러3,라벨러4,라벨러5,라벨러6
0,20200102,'일자리 우수기업 인증제' 개선 고용환경개선비 확대,https://news.naver.com/main/read.nhn?mode=LSD&...,4,4,4,3,4,3
1,20200107,[비즈 칼럼] 일자리 문제 헤쳐 나가려면,https://news.naver.com/main/read.nhn?mode=LSD&...,4,3,4,3,4,4
2,20200108,청년고용률 최고치라는데 청년 체감실업률은 23%,https://news.naver.com/main/read.nhn?mode=LSD&...,4,3,1,2,2,2
3,20200113,사회안전망 강화? '실업급여 단타족'도 늘었다,https://news.naver.com/main/read.nhn?mode=LSD&...,4,3,1,2,2,2
4,20200113,고용보험 가입자 50만명 넘었지만...40%는 60세 이상,https://news.naver.com/main/read.nhn?mode=LSD&...,4,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...
3524,20200831,코로나 충격에 따른 근로 시간 감소_ 과거 위기의 5배,https://news.naver.com/main/read.nhn?mode=LSD&...,2,2,2,2,2,2
3525,20200831,7월 사업체종사자 13만8천명 감소 코로나19 여파에 5개월째 감소,https://news.naver.com/main/read.nhn?mode=LSD&...,4,4,1,2,2,2
3526,20200826,한치 앞도 안보인다..채용문 꽁꽁 닫는 카드사,https://news.naver.com/main/read.nhn?mode=LSD&...,2,2,1,2,2,3
3527,20200831,토스_ 고객 상담직 신입 및 경력 30명 공개 채용,https://news.naver.com/main/read.nhn?mode=LSD&...,4,4,4,4,2,3


In [3]:
search_urls = concatenated_data['URL']

In [4]:
# naver 기사 본문 및 제목 가져오기#
titles = []
contents=[]
dates=[]

# ConnectionError방지
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102" }

for i in tqdm(range(len(search_urls[:5]))):
    original_html = requests.get(search_urls[i],headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    ##########뉴스 제목 가져오기#########
    title = html.select("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2") # F12에서 제목 파트 클릭 후 우클릭 > copy > copy selector 눌러서 나오는 정보 가져오면 됨.
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1,repl='',string=title)
    titles.append(title)
    
    ##########뉴스 날짜 가져오기#########
    date = html.select("#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
    try: #뉴스 날짜가 없는 경우가 있어서 try except로 처리
        date = re.search('date-time="(.+)"',str(date[0])).groups()[0] #정규표현식, 동호가 알려줌.
        dates.append(date) #날짜가 있으면 dates에 추가
    except:
        dates.append('None') #날짜가 없으면 dates에 None 추가

    ###########뉴스 본문 가져오기########
    content = html.select("#dic_area") # F12에서 본문 파트 클릭 후 우클릭 > copy > copy selector 눌러서 나오는 정보 가져오면 됨.
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))
    #html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1,repl='',string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2,'')
    contents.append(content)

100%|██████████| 5/5 [00:00<00:00,  5.20it/s]


In [5]:
print('title의 길이는',len(titles))
print('contents의 길이는',len(contents))
print('dates의 길이는', len(dates))

title의 길이는 5
contents의 길이는 5
dates의 길이는 5


In [ ]:
# 긁어온 데이터와 통합(길이가 같으면 칼럼값으로 정상적으로 들어감)
concatenated_data['content'] = contents

In [49]:
concatenated_data.to_excel("/Users/jongwoom1pro/Coding/text_magician/MJ_newsdata/2020_crawlingNews.xlsx")